In [8]:
import pandas as pd
df = pd.read_csv('runs/results_20200914_124022/india_base/agent_outputs.csv')

In [4]:
import seaborn as sns

year_mask = (df['year'].isin([2020,2030,2040,2050]))
sector_mask = (df['sector_abbr'].isin(['com','res']))
vdf = df.loc[year_mask & sector_mask]
sns.violinplot(data=vdf, x="year", y="payback_period", hue="sector_abbr", split=True)


ValueError: Could not interpret input 'payback_period'

In [106]:
import helper 

# --- Load MNRE data with differentiation by state_name and make long ---
by_state = pd.read_csv(os.path.join('reference_data','Cumulative Installed Capacity by State.csv'))
by_state = by_state.melt(id_vars=['State'])
by_state.columns = ['state_name', 'year','mnre_mw']

# --- Calculate percent by state_name in MNRE by year---
g = by_state.groupby(['state_name','year'])['mnre_mw'].sum()
by_state_pct = g / g.groupby('year').transform('sum')
by_state_pct.name = 'pct_state'
by_state_pct = by_state_pct.reset_index()
by_state = by_state.merge(by_state_pct, on=['year','state_name'])

# --- Read in 2019 adoption by sector and state, excluding some states ---
res = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Residential")
com = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Commercial")
ind = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Industrial")
res['sector_abbr'] = 'res'
com['sector_abbr'] = 'com'
ind['sector_abbr'] = 'ind'
by_sector = pd.concat([res, com, ind], axis='rows')
by_sector.rename({'Unnamed: 0':'state_name', 2019:'sector_mw'}, axis='columns', inplace=True)
by_sector = by_sector[['state_name', 'sector_mw', 'sector_abbr']]

# --- Calcualte percent by state_name and sector ---
g = by_sector.groupby(['state_name','sector_abbr'])['sector_mw'].sum()
by_sector_pct = g / g.groupby(['state_name']).transform('sum')
by_sector_pct.name = 'pct_sector'
by_sector_pct = by_sector_pct.reset_index()

# --- Fill in nans with mean by sector ---
nan_mean_dict = {}
nan_mean_dict['res'] = by_sector_pct.loc[by_sector_pct['sector_abbr'] == 'res', 'pct_sector'].mean()
nan_mean_dict['com'] = by_sector_pct.loc[by_sector_pct['sector_abbr'] == 'com', 'pct_sector'].mean()
nan_mean_dict['ind'] = by_sector_pct.loc[by_sector_pct['sector_abbr'] == 'ind', 'pct_sector'].mean()
null_mask = (by_sector_pct['pct_sector'].isnull())
by_sector_pct.loc[null_mask, 'pct_sector'] = by_sector_pct.loc[null_mask, 'sector_abbr'].map(nan_mean_dict) 

# --- Merge state and sector tables together, and compute compound adoption ---
merged = pd.merge(by_state, by_sector_pct, on=['state_name'], how='outer')
merged.dropna(inplace=True)
merged['pv_capacity_mw'] = merged['pct_sector'] * merged['mnre_mw']

# --- Calculate number of adopters from cumulative capacity ---
system_size_dict = {'res':10, 'com':100, 'ind':200}
merged['pv_system_count'] = merged['pv_capacity_mw'] / merged['sector_abbr'].map(system_size_dict)

# --- Map on state_id ---
merged['state_name'] = merged['state_name'].apply(helper.sanitize_string)
merged['state_id'] = merged['state_name'].map(state_id_lookup)

# --- Clean up and output ---
merged = merged.loc[merged['year'] == merged['year'].max()]
merged = merged[['state_id','pv_capacity_mw','pv_system_count']]
merged.to_csv(os.path.join('india_base','pv_state_starting_capacities.csv'), index=False)

,state_id,pv_capacity_mw,pv_system_count
9,28.0,1109.628768,11.096288
10,28.0,1520.039409,7.600197
11,28.0,456.011823,45.601182
21,12.0,1.581681,0.015817
22,12.0,2.705711,0.013529
...,...,...,...
406,NaN,1.853750,0.009269
407,NaN,7.415000,0.741500
417,34.0,0.000000,0.000000
418,34.0,0.000000,0.000000
